In [ ]:
#OS level tools
import os
import time
import itertools
from collections import defaultdict
from glob import glob
import psutil
from functools import partial
import re
from __future__ import print_function

#array and data structure
import numpy as np
import pandas as pd

#Statistical analysis
import seaborn as sb
from sklearn.metrics import f1_score
from scipy.stats import ttest_1samp, wilcoxon, ttest_ind, mannwhitneyu

#Ipython display and widgets
#import ipywidgets as widgets
from IPython.display import Image, HTML, display
#from IPython.display import clear_output
from IPython.display import Markdown as md

#from ipywidgets import interact_manual

#holoviews and plotting
import holoviews as hv
#import datashader as ds
#from holoviews.operation.datashader import aggregate, shade, datashade
#from holoviews.operation import decimate
#from matplotlib.cm import viridis, jet

#dask parallelization
#import dask.dataframe as dd
from dask import compute, delayed
import dask.threaded
#import dask.multiprocessing

#tsne
from MulticoreTSNE import MulticoreTSNE as TSNE
tsne = TSNE(n_jobs=24)

#color assignment
cmap_fire=['brown', 'maroon', 'red', 'orange', 'yellow', 'white']
cmap_raw=['blue','green','orange', 'red']
cmap_all=['white','white']
cmap_parent=['black','grey']
cmap_pop=(['darkgreen','lightgreen'], ['darkorange','yellow'], ['purple','blueviolet'], ['darkblue','lightblue'], ['indianred','red'])
for i in range(5):
    cmap_pop=cmap_pop+cmap_pop
background = '#D3D3D3'

#export path assignment
#temp_path='/scratch/'+os.environ['USER']+'/'+os.environ['SLURM_JOBID']
export_path="PNG"
#export = partial(export_image, export_path=export_path, background=background)

hv.notebook_extension('bokeh')
display(HTML("<style>.container { width:100% !important;}</style>"))

In [ ]:
hv.opts("RGB [toolbar=None, width=400, height=400, bgcolor='#D3D3D3', fontsize={'title':15, 'xlabel':10, 'ylabel':10, 'ticks':5}]")

In [ ]:
def config_objects(s):
    try:
        with open(s) as config_file:
            config_file.seek(0)
            gates={}
            for line in config_file:
                phenoType=""
                line = line.strip()
                gate = line.split("\t")
                if len(gate)==12:
                    phenoType=gate[11]
                gates.update({"pop"+str(gate[0]):[int(gate[0]), int(gate[1]), int(gate[2]), int(gate[3]), int(gate[4]), int(gate[5]), int(gate[6]), int(gate[7]), int(gate[8]), int(gate[9]), int(gate[10]), phenoType]})
            return gates
    except:
        raise Exception("Error parsing configuration file")
        
def config_summary(s, h):
    try:
        with open(s) as config_file:
            config_file.seek(0)
            gates={}
            for line in config_file:
                phenoType=""
                line = line.strip()
                gate = line.split("\t")
                xmarker=str(h[int(gate[1])-1])
                ymarker=str(h[int(gate[2])-1])
                startx=int((float(gate[3])/200)*4096)
                starty=int((float(gate[5])/200)*4096)
                endx=int((float(gate[4])/200)*4096)
                endy=int((float(gate[6])/200)*4096)
                parent="pop"+gate[7]
                if len(gate)==12:
                    phenoType=gate[11]
                gates.update({"pop"+str(gate[0]):[int(gate[0]), parent, xmarker, ymarker, phenoType, startx, endx, starty, endy]})
            return gates
    except:
        raise Exception("Error parsing configuration file")
        
_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]   

def natural_sort(l): 
    #https://stackoverflow.com/a/4836734/846892
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    def atoi(text):
        return int(text) if text.isdigit() else text

    return [atoi(c) for c in re.split('(\d+)', text)]

def label_color (pops, row):
    eventcolor=0
    for i, pop in enumerate(pops):
        if row[pop]==0:
            eventcolor=i+1
    return eventcolor

def label_color2 (pops, row):
    eventcolor="base"
    for i, pop in enumerate(pops):
        if row[pop]==0:
            eventcolor=pop
    return eventcolor

def parseCohort(s):
    cohort_file=open(s)
    
    return

def parseDataFrame(s):
    result_file=open(s)
    sampleLabel=os.path.splitext(s)[0]
    events = sum(1 for line in result_file) -1 #quickly determine number of events
    result_file.seek(0) #rewind file to beginning
    header = result_file.readline()
    header = header.strip()
    headers = header.split("\t")
    
    pop_offset=len(headers)
    popList=[]
    for i,header in enumerate(headers):
        if header == "pop1":
            pop_offset=i
        if "pop" in header:
            popList.append(header)
    markers = headers[0:pop_offset]
    result_file.seek(0) #rewind file to beginning
    
    df = pd.read_csv(s, sep='\t')
    dataIndex={}
    for i,header in enumerate(headers):
        dataIndex.update({header:i})
    df['pop0']=0
    return [sampleLabel,headers,markers,popList,df]

def parseDAFi(s):
    df = pd.read_csv(s, sep='\t')
    df['pop0']=0
    return df

def html_row(file):
     return '<img src="{}" style="display:inline;margin:1px" title="{}"/>'.format(export_path+"/"+file+".png",file,file)

# Summary Tables and Statistics Report

In [ ]:
titlefile=open('description.txt')
title=titlefile.readline()
titlefile.close()
display(md("# Dataset: %s"%(title)))

## Metadata

In [ ]:
%%output backend='bokeh'
%%opts Table [width=1200]
metadatafilename='metadata.txt'
metadatafile=open(metadatafilename)
metaheader = metadatafile.readline()
metaheader = metaheader.strip()
metaheaders = metaheader.split("\t")
metaDf=pd.read_csv('metadata.txt', sep='\t')
metaTable=hv.Table(metaDf)
display(metaTable)

In [ ]:
%%output backend='bokeh'
%%opts Table [width=1200]
#hv.Table(metaDf.loc[(metaDf['Gender']=='F') & (metaDf['Age']>40)])

In [ ]:
gatedFiles=sorted(glob('Gated/*/flock*.txt'))
gatedDelayed=[[(os.path.split(os.path.dirname(fn))[1]),delayed(parseDAFi)(fn)] for fn in gatedFiles]
sample_labels=[os.path.split(os.path.dirname(fn))[1] for fn in gatedFiles]
dfArray=compute(*gatedDelayed, get=dask.threaded.get)

In [ ]:
bisectFiles=sorted(glob('Bisect/*/flock*.txt'))
if bisectFiles:
    bisectDelayed=[[(os.path.split(os.path.dirname(fn))[1]),delayed(parseDAFi)(fn)] for fn in bisectFiles]
    bisectArray=compute(*bisectDelayed, get=dask.threaded.get)

In [ ]:
headers=list(dfArray[0][1])
pop_offset=len(headers)
popList=[]
for i,header in enumerate(headers):
    if header == "pop1":
        pop_offset=i
    if "pop" in header:
        popList.append(header)
markers = headers[0:pop_offset]

## DAFi Configuration

In [ ]:
%%output backend='bokeh'
%%opts Table.gates [width=1200]
%%opts Table.summary [width=1200]
configLabel="pipeline.config"
gates=config_objects(configLabel)
num_gates = len(gates)
summary=config_summary(configLabel, headers)
num_gates = len(summary)

gatesummary = [v for v in summary.values()]
di = {summary.get(element)[0]:str(summary.get(element)[0]).zfill(2)+"_"+summary.get(element)[4] for i,element in enumerate(summary)}
summaryTable=hv.Table(gatesummary,kdims=['Population','Parent','XMarker','YMarker','phenotype','startx', 'endx', 'starty', 'endy'], group='summary', label='Summary')

sortedTable=summaryTable.sort('Population')
sortedTable

## Marker and Axis Tables

In [ ]:
%%output backend='bokeh'
axis_popIndexDict = defaultdict(list)
popBounds={}
axises=[]
composite_axis=0
last_xmarker=""
last_ymarker=""
last_parent=0
gatesconfig=[]
for i in range(len(gates)):
    pop="pop"+str(i+1)
    config=gates.get(pop)
    xmarker=str(headers[config[1]-1])
    ymarker=str(headers[config[2]-1])
    startx=int((float(config[3])/200)*4096)
    starty=int((float(config[5])/200)*4096)
    endx=int((float(config[4])/200)*4096)
    endy=int((float(config[6])/200)*4096)
    parent=int(config[7])
    ctype=int(config[8])
    popBounds.update({pop:[xmarker, ymarker, startx,starty,endx,endy,ctype,"pop"+str(parent)]})
    key="axis"+str(composite_axis).zfill(2) 
    if (xmarker != last_xmarker) or (ymarker != last_ymarker) or (parent != last_parent):
        composite_axis=composite_axis+1
        key="axis"+str(composite_axis).zfill(2)
        axises.append([xmarker,ymarker,key,"pop"+str(parent)])
    axis_popIndexDict[key].append(pop)
    last_xmarker=xmarker
    last_ymarker=ymarker
    last_parent=parent
    gatesconfig.append([pop,xmarker,ymarker,parent])
poplist=natural_sort(popBounds.keys())
num_axises = len(axises)
markerTable=hv.Table(markers,kdims=['Marker'])
axisTable=hv.Table(axises,kdims=['Xmarker','Ymarker','Axis Index', 'Parent Pop'])
axis_popTable=hv.Table(axis_popIndexDict, kdims=['Axis Index'], vdims=['sub populations'])
display(markerTable+axisTable+axis_popTable.sort('Axis Index'))

In [ ]:
hv.notebook_extension('matplotlib')

## Dataset Batch Population Statistics

### Manual Gated Population Percentage and Events Tables

In [ ]:
%%output backend='bokeh'
%%opts Table.gates [width=len(sample_labels)*100]
%%opts Table.summary [width=len(sample_labels)*100]

if bisectFiles:
    b_batchpercent_df = pd.read_csv('Bisect/Batch_percentages.txt', sep='\t', index_col=0)
    b_batchevents_df = pd.read_csv('Bisect/Batch_events.txt', sep='\t', index_col=0)
    display(hv.Layout(hv.Table(b_batchpercent_df,kdims=sample_labels, group='summary', label='Summary')+hv.Table(b_batchevents_df,kdims=sample_labels, group='summary', label='Summary')).cols(1))
else:
    print ("Bisection files not available")

### DAFi Gated Population Percentage and Events Tables

In [ ]:
%%output backend='bokeh'
%%opts Table.gates [width=len(sample_labels)*100]
%%opts Table.summary [width=len(sample_labels)*100]

batchpercent_df = pd.read_csv('Gated/Batch_percentages.txt', sep='\t', index_col=0)

batchevents_df = pd.read_csv('Gated/Batch_events.txt', sep='\t', index_col=0)

display(hv.Layout(hv.Table(batchpercent_df,kdims=sample_labels, group='summary', label='Summary')+hv.Table(batchevents_df,kdims=sample_labels, group='summary', label='Summary')).cols(1))

### Combined Percent/Events Dataframe

In [ ]:
%%output backend="bokeh"
%%opts Table [width=1000]

p_df=pd.DataFrame(batchpercent_df.unstack())
p_df.columns=['Percent']

e_df=pd.DataFrame(batchevents_df.unstack())
e_df.columns=['Events']

if bisectFiles:
    bp_df=pd.DataFrame(b_batchpercent_df.unstack())
    bp_df.columns=['Bisect Percent']
    be_df=pd.DataFrame(b_batchevents_df.unstack())
    be_df.columns=['Bisect Events']
    
    c_df=pd.concat([p_df,e_df,bp_df,be_df],axis=1, join='outer').reset_index()
    c_df.columns=['Sample','Population','Percent','Events', 'Bisect Percent', 'Bisect Events']
    c_df=c_df.replace({"Population":di})
else:
    c_df=pd.concat([p_df,e_df],axis=1, join='outer').reset_index()
    c_df.columns=['Sample','Population','Percent','Events']
    c_df=c_df.replace({"Population":di})

mainDf=pd.merge(c_df, metaDf, on='Sample', how='inner')

display(hv.Table(mainDf))

In [ ]:
metavaluecols=list(metaDf.select_dtypes(include=['int64','float64']))
metastrcols=list(metaDf.select_dtypes(include=['object']))
categorylist=[]
samplesByCat={}
metavalueranges=[]
if len(metastrcols):
    
    for col in metastrcols:
        if len(set(mainDf[str(col)]))<10:
            categorylist.append(str(col))
    if len(categorylist):
        
        for col in categorylist:
            tempset=set(mainDf[str(col)])
            for element in tempset:
                samplesByCat.update({str(element):list(set(mainDf.loc[mainDf[str(col)]==str(element)]['Sample']))})

if len(metavaluecols):
    for col in metavaluecols:
        metavalueranges.append([(mainDf[metavaluecols[0]].min(),mainDf[metavaluecols[0]].mean()),(mainDf[metavaluecols[0]].mean(),mainDf[metavaluecols[0]].max())])

In [ ]:
#mainDf[mainDf['Sample'].isin(samplesByCat.get('F'))]

### Box Plots

In [ ]:
%%output backend="bokeh" size=200
%%opts BoxWhisker [xrotation=45]
if bisectFiles:
    b_percentBoxPlot=hv.BoxWhisker(mainDf, kdims=['Population'],vdims='Bisect Percent').relabel('Manual Gated Population Percentages')
    b_eventsBoxPlot=hv.BoxWhisker(mainDf, kdims=['Population'],vdims='Bisect Events').relabel('Manual Gated Population Events')
    display(b_percentBoxPlot+b_eventsBoxPlot)

In [ ]:
%%output backend="bokeh" size=200
%%opts BoxWhisker [xrotation=45]
percentBoxPlot=hv.BoxWhisker(mainDf, kdims=['Population'],vdims='Percent').relabel('DAFi Gated Population Percentages')
eventsBoxPlot=hv.BoxWhisker(mainDf, kdims=['Population'],vdims='Events').relabel('DAFi Gated Population Events')
display(percentBoxPlot+eventsBoxPlot)

### Bar Plots 

In [ ]:
%%output backend="bokeh"
%%opts Bars [bgcolor='pink' width=600]
if bisectFiles:
    bisectPercentCV=hv.Bars((b_batchpercent_df.transpose().std()/b_batchpercent_df.transpose().mean())*100,'Popuation','Coefficient of Variation').relabel('Manual Gated Population Percentages CV')
    bisectEventCV=hv.Bars((b_batchevents_df.transpose().std()/b_batchevents_df.transpose().mean())*100,'Popuation','Coefficient of Variation').relabel('Manual Gated Population Events CV')
    #display(bisectPercentCV+bisectEventCV)

In [ ]:
%%output backend="bokeh"
%%opts Bars [bgcolor='yellow' width=600]
clusterPercentCV=hv.Bars((batchpercent_df.transpose().std()/batchpercent_df.transpose().mean())*100,'Popuation','Coefficient of Variation').relabel('DAFi Gated Population Percentages CV')
clusterEventCV=hv.Bars((batchevents_df.transpose().std()/batchevents_df.transpose().mean())*100,'Popuation','Coefficient of Variation').relabel('DAFi Gated Population Events CV')
#display(clusterPercentCV+clusterEventCV)

In [ ]:
%%output backend="bokeh"
%%opts Bars [bgcolor='lightgrey' width=600]
%%opts Bars (alpha=0.5 color=Cycle('Category10'))
%%opts Overlay [legend_position="bottom"]
if bisectFiles: 
    display(clusterPercentCV*bisectPercentCV+clusterEventCV*bisectEventCV)

## F-measure (comparing DAFi gating results to manual gating)

In [ ]:
if bisectFiles:
    FmeasureDict={(sample, k+1):f1_score(bisectArray[j][1][pop], dfArray[j][1][pop], average='macro') for j, sample in enumerate(sample_labels) for k, pop in enumerate(poplist)}
    fscore_data = list(map(list, zip(*FmeasureDict.keys()))) + [FmeasureDict.values()]
    fscore_df = pd.DataFrame(list(zip(*fscore_data))).set_index([0, 1])[2].unstack().transpose()
    fscore_df = pd.DataFrame(fscore_df.unstack())
    fscore_df.columns=["Fscore"]
    fscore_df = fscore_df.reset_index()
    fscore_df.columns=["Sample","Population","Fscore"]
    fscore_df=fscore_df.replace({"Population":di})
else:
    print ("Manual gating files not available")

### F-measure Table

In [ ]:
%%output backend='bokeh'
%%opts Table.fscore [width=1000]

if bisectFiles:
    fscore_Table=hv.Table(fscore_df, kdims=['Sample','Population'],group='fscore')
    display(fscore_Table)

### F-measure Box Plot

In [ ]:
%%output backend="bokeh" size=300
%%opts BoxWhisker [xrotation=45]
if bisectFiles:
    fscore_boxPlot=hv.BoxWhisker(fscore_df, kdims=['Population'],vdims='Fscore').relabel('F-measure Statistics')
    display(fscore_boxPlot)
else:
    print ("Manual gating files not available")

In [ ]:
%%output backend="bokeh"
%%opts Scatter [width=1200 height=600 show_grid=True tools=['hover']]
%%opts Scatter (color=Cycle('Category20') size=10 alpha=0.8 line_color='k')
%%opts NdOverlay [legend_position='bottom' show_frame=False]
if bisectFiles:
    fscore_scatter = fscore_Table.to.scatter('Population', 'Fscore')
    fscore_plot=fscore_scatter.overlay('Sample')
    display((fscore_boxPlot*fscore_plot).relabel("Fmeasure Box Plot with Sample Overlay"))
else:
    print ("Manual gating files not available")

## tSNE mapping of population percentage

In [ ]:
batchpercent_df=batchpercent_df.rename(di).round(2)
batchevents_df=batchevents_df.rename(di)

In [ ]:
percentdf=batchpercent_df.transpose()
tsne_data_array=percentdf.values.astype(np.float64)
data_tsne = tsne.fit_transform(np.copy(tsne_data_array))
dfn=pd.DataFrame(data_tsne, columns=['tsne-x','tsne-y'], index=percentdf.index)
results=pd.concat([percentdf,dfn],axis=1)
colnames=list(results)[0:-2]

In [ ]:
%%output backend='bokeh'
%%opts Scatter.tSNE_byPop (size=5 nonselection_color='grey' cmap='Reds') [bgcolor='#D3D3D3' color_index=2 width=400 height=400 tools=['hover','box_select','poly_select']] 
%%opts Layout [shared_datasource=True]
labels=[kd for i, kd in enumerate(colnames)]
holomap = hv.HoloMap({(kd): hv.Scatter(results, kdims=['tsne-x','tsne-y'],vdims=[kd], group="tSNE_byPop") for i, kd in enumerate(colnames)}, kdims='Population')
hv.Layout(holomap.layout())

In [ ]:
display(HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>'''))